In [2]:
import os
from typing import Any

from dotenv import load_dotenv
from utils.agent_visualizer import print_activity

from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient

# 02 - 可观测性智能体

在之前的笔记本中，我们构建了一个基本的研究智能体和一个主助理多智能体框架。虽然我们构建的智能体已经很强大，但它们在可以做什么方面仍然受到限制：网络搜索智能体仅限于搜索互联网，我们的主助理智能体仅限于与其自己的文件系统交互。

这是一个严重的限制：现实世界的智能体通常需要与数据库、API、文件系统和专门服务等其他系统进行交互。[MCP (模型上下文协议)](https://modelcontextprotocol.io/docs/getting-started/intro)是一个用于AI工具集成的开源标准，允许我们的智能体与这些外部系统之间轻松连接。在本笔记本中，我们将探索如何将MCP服务器连接到我们的智能体。

**需要更多MCP详细信息？**有关综合设置说明、配置最佳实践和故障排除提示，请参阅[Claude Code MCP文档](https://docs.claude.com/en/docs/claude-code/mcp)。

## MCP服务器介绍
### 1. Git MCP服务器

首先让我们赋予我们的智能体理解和处理Git仓库的能力。通过将[Git MCP服务器](https://github.com/modelcontextprotocol/servers/tree/main/src/git)添加到我们的智能体，它可以获得13个特定于Git的工具的访问权限，这些工具让它检查提交历史、检查文件更改、创建分支甚至进行提交。这将我们的智能体从开发工作流程中的被动观察者转变为主动参与者。在本示例中，我们将配置智能体仅使用Git工具来探索仓库的历史。这很简单，但知道这一点后，不难想象可以自动创建拉取请求、分析代码演进模式或帮助管理跨多个仓库的复杂Git工作流的智能体。

In [ ]:
# define our git MCP server (it was downloaded when you ran uv sync as it is defined in the pyproject.toml file)
git_mcp: dict[str, Any] = {
    "git": {
        "command": "uv",
        "args": ["run", "python", "-m", "mcp_server_git", "--repository", os.getcwd()],
    }
}

In [4]:
messages = []
async with (
    ClaudeSDKClient(
        options=ClaudeAgentOptions(
            model="claude-sonnet-4-5",
            mcp_servers=git_mcp,
            allowed_tools=[
                "mcp__git"
            ],  # For MCP tools, in allowed tools we must add the mcp__serverName__toolName format or mcp__serverName to enable all
            permission_mode="acceptEdits",  # auto-accept file edit permissions
        )
    ) as agent
):
    await agent.query(
        "Use ONLY your git mcp tools to quickly explore this repo's history and gimme a brief summary."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__git()
✓ Tool completed
🤖 Thinking...
🤖 Using: Bash()
🤖 Using: Bash()
🤖 Using: Bash()
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

### 2. GitHub MCP服务器

现在让我们从本地Git操作升级到完整的GitHub平台集成。通过切换到[官方GitHub MCP服务器](https://github.com/github/github-mcp-server/tree/main)，我们的智能体可以访问与GitHub整个生态系统交互的100多个工具——从管理问题和拉取请求到监控CI/CD工作流和分析代码安全警报。该服务器可以与公共和私有仓库配合使用，让您的智能体能够自动化通常需要多个手动步骤的复杂GitHub工作流。

#### 第1步：设置您的GitHub令牌

您需要一个GitHub个人访问令牌。在[这里](https://github.com/settings/personal-access-tokens/new)获取一个，并放在.env文件中，格式为```GITHUB_TOKEN="<token>"```
> 注意：获取令牌时，选择具有默认选项的"细粒度"令牌（即，公共仓库，无账户权限），这是让此演示工作的最简单方法。

此外，对于此示例，您必须在机器上运行[Docker](https://www.docker.com/products/docker-desktop/)。需要Docker是因为GitHub MCP服务器在容器化环境中运行以确保安全和隔离。

**Docker快速设置：**
- 从[docker.com](https://www.docker.com/products/docker-desktop/)安装Docker Desktop
- 确保Docker正在运行（您将在系统托盘中看到Docker图标）
- 在终端中使用`docker --version`进行验证
- **故障排除：**如果Docker无法启动，请检查您的BIOS中是否启用了虚拟化。有关详细设置说明，请参阅[Docker文档](https://docs.docker.com/get-docker/)

#### 第2步：定义mcp服务器并启动智能体循环！

In [6]:
# define our github mcp server
load_dotenv(override=True)
github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

In [7]:
# run our agent
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",  # auto-accept permissions
    )
) as agent:
    await agent.query(
        "Use ONLY your GitHub MCP tools to search for the anthropics/claude-agent-sdk-python repository and and give me a couple facts about it"
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__github__search_repositories()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_file_contents()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

## 真实用例：可观测性智能体

现在，通过如此简单的设置，我们已经可以让智能体充当自我修复软件系统！

In [9]:
load_dotenv(override=True)

prompt = """Monitor the GitHub Actions workflows for facebook/react.
Look at the last triggered CI pipeline. 
1. Analyze the trigger for the pipeline
2. Identify whether the pipeline passed or not
3. If it failed, explain which test failed
4. Identify whether human involvement is required

IMPORTANT: Do not raise a PR, issue, or bug on github yet. Just give me a summary of your findings and plan.

Focus on the 'CI' workflow specifically. Use your Github MCP server tools!"""

github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",
    )
) as agent:
    await agent.query(prompt)
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__list_workflows()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_runs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_workflow_run()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_jobs()
✓ Tool completed
🤖 Thinking...
🤖 Using: WebFetch()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(f"\nResult:\n{messages[-1].result}")

### 将可观测性智能体作为模块使用

`observability_agent/agent.py`文件包含与研究智能体或主助理智能体相同的最小辅助函数，只是针对GitHub监控进行了增强。

和之前一样，要将其作为模块在Python代码中使用：

In [ ]:
from observability_agent.agent import send_query

result = await send_query(
    "Check the CI status for the last 2 runs in anthropics/claude-agent-sdk-python. Just do 3 tool calls, be efficient."
)
print(f"Monitoring result: {result}")

我们也可以与这个智能体进行多轮对话：

In [ ]:
# Example 2: Multi-turn conversation for deeper monitoring
result1 = await send_query("What's the current CI status for facebook/react?")
print(f"Initial check: {result1[:250]}...\n")

In [ ]:
# Continue the conversation to dig deeper
result2 = await send_query(
    "Are there any flaky tests in the recent failures? You can only make one tool call.",
    continue_conversation=True,
)
print(f"Follow-up analysis: {result2[:250]}...")

## 结论

我们演示了Claude Code SDK如何通过模型上下文协议(MCP)实现与外部系统的无缝集成。从通过Git MCP服务器的本地Git操作开始，我们逐步扩展到对100多个GitHub特定工具的完整GitHub平台集成。这将我们的智能体从本地助手转变为强大的可观测性系统，能够监控工作流、分析CI/CD失败并为生产系统提供可操作洞察。

通过将MCP服务器连接到我们的智能体，我们创建了一个自主可观测性系统，可以监控GitHub Actions工作流、区分真实失败和安全限制，并提供测试失败的详细分析。该系统展示了智能体如何主动参与您的DevOps工作流，从被动监控转向智能事件响应。

到目前为止，这结束了我们通过Claude Code SDK教程系列的旅程。我们从简单的研究智能体发展到复杂的多智能体编排，最终通过MCP进行外部系统集成。总的来说，这些模式为构建能够处理现实世界复杂性同时保持治理、合规性和可观测性的生产就绪智能体系统提供了基础。

### 您在整个笔记本中学到的内容

**来自笔记本00（研究智能体）**
- 使用`query()`和`ClaudeSDKClient`的核心SDK基础知识
- 使用WebSearch和Read的基本工具使用
- 简单的智能体循环和对话管理

**来自笔记本01（主助理）**
- 高级功能：记忆、输出样式、计划模式
- 通过子智能体进行多智能体协调
- 通过钩子和自定义命令进行治理
- 企业级智能体架构

**来自笔记本02（可观测性智能体）**
- 通过MCP服务器进行外部系统集成
- 实时监控和事件响应
- 生产工作流自动化
- 可扩展的智能体部署模式

所有三个智能体的完整实现都在各自的目录中（`research_agent/`、`chief_of_staff_agent/`、`observability_agent/`），准备好作为集成到您生产系统中的灵感。